# Classification

## Install packages

In [ ]:
!pip install asent
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install gensim

## Import packages

In [2]:
from asent import lexicons
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import gensim.downloader
from sklearn.linear_model import LogisticRegression

## Load and split data

The data is loaded using the [```asent```]("https://github.com/KennethEnevoldsen/asent") package. It contains 7504 words and a continuous sentiment label which is constructed by [two annotaters]("https://ojs.aaai.org/index.php/ICWSM/article/view/14550").

I have split the data into a training and a test set containing 80% and 20% of the data, respectively. It is important that we all use the same random state, to ensure we get the same split. That way, we can compare our models on the held-out test set next week.

In [3]:
lex = pd.DataFrame(lexicons.get("lexicon_en_v1").items(), columns=["word", "sentiment"])

train, test = train_test_split(lex, test_size=0.2, random_state=42)

## Preprocessing and feature generation

To make the task a bit simpler, we binarise the sentiment label. We consider all words with a sentiment score above 0 as positive (1) and all words with a sentiment score below 0 as negative (0).

In [4]:
y = [1 if x>0 else 0 for x in train["sentiment"]]

In [5]:
print(train["sentiment"].to_list())
print(y)

[1.5, 1.3, 2.8, 0.8, 0.6, 0.1, -2.1, 0.5, -1.0, 2.2, 2.3, 3.0, -1.9, 2.1, -1.9, -0.1, -0.9, -1.6, -1.8, 2.4, -2.8, 1.4, -2.1, -1.9, -1.0, -3.4, 1.3, -1.6, 0.3, -2.1, -1.2, -1.5, 1.1, 2.8, 1.6, -1.7, -2.3, 0.8, 0.1, -2.8, -2.3, -1.6, -0.9, -1.0, 1.9, 1.5, 1.9, -1.6, 2.6, 2.6, -0.8, -1.6, -2.1, 2.2, -0.5, 3.1, 1.8, 1.6, 0.7, 0.3, -0.2, -1.3, -2.4, -0.8, -1.2, -2.6, 1.8, -0.9, 1.8, -1.1, -2.2, 1.5, 2.8, -1.2, -1.5, -1.2, 2.9, 1.6, 0.4, -2.2, -1.2, 1.3, 2.2, 1.9, 2.3, -2.7, 2.2, -1.2, -1.2, 0.2, -1.8, -2.9, -1.6, -1.7, 0.5, 2.1, 1.6, 2.5, 1.4, 0.5, 0.9, 0.8, -0.1, 1.8, -0.9, -2.1, -3.1, -2.0, -2.4, 0.6, 1.3, 1.6, 0.9, 1.8, -2.1, 1.9, 1.6, -1.8, 1.2, -1.2, -1.6, -1.6, -1.0, -2.2, 2.0, -0.2, -1.4, 2.0, -1.3, 1.7, -1.9, -1.9, 2.2, -1.0, -1.6, -1.1, 0.1, 1.1, 1.7, -2.0, 1.2, -1.7, 2.7, -0.5, -0.9, -1.5, 1.6, 1.1, -1.3, -2.7, 1.2, 2.2, -0.4, -1.2, -0.8, -2.1, 1.4, 1.1, -1.3, -2.0, 1.3, -1.2, 1.3, 0.2, -0.7, -2.1, -1.6, 1.0, -1.9, -1.0, 1.9, 1.6, -1.2, 0.1, 1.5, -1.9, -0.9, 1.5, -2.0, -2.3, -1.3

Now we have our labels for training, and we can start selecting what features to use to predict the sentiment of the words. I'm going to use the ```glove``` embeddings from earlier, but feel free to use any other embeddings or features you think will work well.

In [6]:
embeddings = gensim.downloader.load("glove-wiki-gigaword-300")

Some words are not in the embedding model vocabulary, so we need to decide how to represent them instead. I'm going to use a zero vector, but another common approach is to use the average of the embeddings of all other words (mean imputation).

In [7]:
features = [embeddings[r["word"]] if r["word"] in embeddings.index_to_key else np.zeros(shape=300) for i, r in train.iterrows()]

In [ ]:
features

Now we just need to transform the features to a format that the classifier can use. I'm going to use a simple logistic regression model from ```sklearn```, but you can use any other classifier you think will work well.

In [10]:
X = np.array(features)

In [11]:
clf = LogisticRegression(random_state=42)

Once you have specified your model, you can fit it to the training data.

In [12]:
clf.fit(X, y)

LogisticRegression(random_state=42)

The class has a ```score()``` method that takes features and true labels and returns an accuracy score. It uses the fitted model to predict labels based on the features and compares them to the true labels.

In [13]:
clf.score(X, y)

0.7881059470264867

Now you have a sentiment classifier! If there indeed is a strong relationship between the sentiment of a word and its embedding, the relationships learned during training should generalise to the test set, which we will test next week.

For now, you can iterately improve the model by tweaking different parts of the pipeline, and reevaluating the performance on the training set. Here are some parameters you can try to change:

- try embedding models trained on different data (e.g., you can get an overview of different embeddings models that are available through the gensim api by running ```gensim.downloader.info()["models"].keys()```)
- try mean imputation
- try to change the parameters of the model (you can find an overview of the parameters in the [sklearn documentation]("https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html"))
- try different models (you can find an overview of the sklearn supervised learning library [here]("https://scikit-learn.org/stable/supervised_learning.html"))
- try to use the continuous sentiment scores as labels - can you get similar performance? (why/why not?)